In [ ]:
# System 
import os
import sys 

# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# Datasets
from keras.datasets import cifar10
import pickle

# Show images 
from matplotlib import pyplot
from scipy.misc import toimage

In [ ]:
# Set image dimension ordering
K.set_image_dim_ordering('tf')

# Fix random seed
seed = 7
numpy.random.seed(seed)

In [ ]:
# Load data

# Data file 
cifar10_pkl = 'cifar10.pkl' 

# Save to pkl
if not os.path.isfile(cifar10_pkl): 
    # Downloaded from: http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    with open(cifar10_pkl, 'wb') as f:
        pickle.dump(((x_train, y_train), (x_test, x_test)), f)
else: 
    # Read from pkl
    with open(cifar10_pkl, 'rb') as f:
        (x_train, y_train), (x_test, y_test) = pickle.load(f)

In [ ]:
# Create a grid of 3x3 images
for i in range(0, 9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(toimage(x_train[i]))
# show the plot
pyplot.show()

In [ ]:
# Normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# One hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# Create model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [ ]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=32)

# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Serialize model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# Serialize weights to HDF5
model.save_weights("model.hdf5")